In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# Load the data
df = pd.read_csv("LoanStats_2019Q1.csv")

# Preview the data
df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [3]:
lae = LabelEncoder()
lae.fit(df["home_ownership"])
df["home_ownership"] = lae.transform(df["home_ownership"])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [4]:
# Create our features
X= df.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our target
y=df["loan_status"]


In [5]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [7]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape


(51612, 92)

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)


In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)


BalancedRandomForestClassifier(random_state=1)

In [12]:
# Calculated the balanced accuracy score

y_pred = brf.predict(X_test_scaled)

balanced_accuracy_score(y_test, y_pred)


0.7799029857661425

In [13]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   59,    28],
       [ 2026, 15092]])

In [14]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.88      0.05      0.77      0.59        87
   low_risk       1.00      0.88      0.68      0.94      0.77      0.61     17118

avg / total       0.99      0.88      0.68      0.93      0.77      0.61     17205



In [15]:
# List the features sorted in descending order by feature importance

importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.07689103353204897, 'total_rec_prncp'),
 (0.056660286941882854, 'last_pymnt_amnt'),
 (0.05471256284512414, 'total_pymnt_inv'),
 (0.052837645053335235, 'total_pymnt'),
 (0.052637625207642896, 'total_rec_int'),
 (0.03281030139654061, 'int_rate'),
 (0.022326981068981323, 'bc_util'),
 (0.019217155960431158, 'dti'),
 (0.018809141906592292, 'max_bal_bc'),
 (0.018786938924772706, 'out_prncp_inv'),
 (0.01695333129105188, 'mths_since_recent_inq'),
 (0.016477874102464044, 'tot_cur_bal'),
 (0.01630361957355067, 'il_util'),
 (0.016079740496160743, 'annual_inc'),
 (0.015999256935088156, 'issue_d_Jan-2019'),
 (0.015981379116332402, 'bc_open_to_buy'),
 (0.01589807782588542, 'out_prncp'),
 (0.015168585112231374, 'installment'),
 (0.01498190985450308, 'mo_sin_old_rev_tl_op'),
 (0.014318175276629928, 'total_bal_il'),
 (0.01420276053897031, 'total_il_high_credit_limit'),
 (0.013789406133749613, 'total_acc'),
 (0.013319850829036235, 'revol_bal'),
 (0.013188719425542427, 'avg_cur_bal'),
 (0.013187980646

In [16]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [17]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7799029857661425

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)


array([[   59,    28],
       [ 2026, 15092]])

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.88      0.05      0.77      0.59        87
   low_risk       1.00      0.88      0.68      0.94      0.77      0.61     17118

avg / total       0.99      0.88      0.68      0.93      0.77      0.61     17205



### Final Questions

Q.Which model had the best balanced accuracy score?
Answer:- Easy Ensemble Classifier with score of approximately 0.9254 compared to RandomForestClassifier score of approx 0.6721

Q.Which model had the best recall score?
Answer:- Random Forest Classifier had the best score of 1.

Q.Which model had the best geometric mean score?
Answer:-Easy Ensemble Classifier had the best geometric mean score of 0.93

Q.What are the top three features?
Answer:- The top three features are:
'total_rec_prncp'

'total_pymnt'

'total_pymnt_inv'